In [1]:
import pandas as pd
import numpy as np
np.random.seed(42)
import matplotlib.pyplot as plt, numpy as np
from sklearn import metrics
import pickle
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.pipeline import make_pipeline
from sklearn.compose import ColumnTransformer

In [2]:
data = pd.read_excel('data_copy.xlsx')

In [3]:
data

,State,Market,Market Size,Profit,Margin,Sales,COGS,Total Expenses,Marketing,Inventory,...,Budget COGS,Budget Margin,Budget Sales,Product Type,Product,Type,Sale_Year,Sale_Month,Sale_Day,Clusters
0,Connecticut,East,Small Market,107,176,292,116,69,38,962,...,110,160,270,Coffee,Columbian,Regular,2010,4,1,3
1,Connecticut,East,Small Market,75,135,225,90,60,29,1148,...,80,130,210,Coffee,Columbian,Regular,2010,7,1,1
2,Connecticut,East,Small Market,122,195,325,130,73,42,1134,...,110,180,290,Coffee,Columbian,Regular,2010,11,1,3
3,Connecticut,East,Small Market,105,174,289,115,69,37,1166,...,100,160,260,Coffee,Columbian,Regular,2010,12,1,1
4,Connecticut,East,Small Market,104,135,223,90,56,29,1148,...,80,130,210,Coffee,Columbian,Regular,2011,7,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2966,Louisiana,South,Small Market,62,107,206,86,65,32,484,...,80,100,180,Herbal Tea,Lemon,Decaf,2011,2,1,4
2967,Louisiana,South,Small Market,43,93,167,76,58,28,613,...,70,90,160,Herbal Tea,Lemon,Decaf,2011,7,1,4
2968,Louisiana,South,Small Market,37,83,160,67,58,25,599,...,60,80,140,Herbal Tea,Lemon,Decaf,2011,9,1,0
2969,Louisiana,South,Small Market,1,87,150,63,86,57,37,...,50,70,120,Espresso,Caffe Latte,Regular,2010,4,1,4


In [4]:
x = data[['State', 'Market', 'Market Size', 'Product Type', 'Type']]
y = data['Product']

In [5]:
from sklearn.preprocessing import OneHotEncoder
nom_cols = ['State', 'Market', 'Market Size', 'Product Type', 'Type']
nominal_pipeline = make_pipeline(OneHotEncoder())

# stack your pipelines in column transformer
transformer = ColumnTransformer(transformers=[('nominal' ,nominal_pipeline,nom_cols)])
# Now lets fit and transform the data
transformed = transformer.fit_transform(x)

In [6]:
# create train and test data
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size=0.3, random_state=42)

In [7]:
# adding GradientBoostingClassifier to transformer pipeline
final_pipeline = make_pipeline(transformer, GradientBoostingClassifier())

# fit the data to new pipeline & model
final_pipeline.fit(x_train, y_train)

# check the accuracy of our model
final_pipeline.score(x_test, y_test)

0.6378923766816144

In [8]:
final_pipeline

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('nominal',
                                                  Pipeline(steps=[('onehotencoder',
                                                                   OneHotEncoder())]),
                                                  ['State', 'Market',
                                                   'Market Size',
                                                   'Product Type', 'Type'])])),
                ('gradientboostingclassifier', GradientBoostingClassifier())])

In [16]:
# filename = 'final_pipeline.pkl' # create a variable with name you wanna give to the file
# pickle.dump(final_pipeline, open(filename,'wb')) 

In [12]:
# def segment_customers(input_data):
    
#     prediction=final_pipeline.predict(pd.DataFrame(input_data, columns=['State', 'Market', 'Market Size', 'Product Type', 'Type']))
#     print(prediction)
#     return prediction

In [14]:
# State = "California"
# Market = 'East'
# Market_Size = 'Small Market'
# Product_Type = "Coffee"
# Type = 'Regular'
# result=segment_customers([[State,Market,Market_Size,Product_Type,Type]])


['Columbian']


In [ ]:
import streamlit as st
import pandas as pd
# import pickle

# Set the title of the app
st.title("Market Segmentation App")

from PIL import Image
st.subheader("Hello!    Welcome to Product Recommendation")

image = Image.open("MarketSegment.png")
st.image(image,use_column_width = True)

text = "<b>This app is designed to help businesses decide which product they should focus on to sell more. The app recommends which product out of the 13 categories of products named Columbian, Lemon, Decaf Espresso, Darjeeling, Chamomile, Early Grey, Caffe Mocha, Decaf Irish Cream, Caffe Latte, Regular Espresso, Mint, Amaretto and Green Tea will be most likely to sell based on certain factors. These factors include the State, Market, Market Size, Product Type, and Type. The app's prediction accuracy is approximately 75%, making it a reliable tool for businesses to plan their inventory, make decisions regarding offers, and determine the price of the product. By entering the relevant details, businesses can receive data-driven suggestions that will help them increase their sales and profitability.</b>"
st.write(text, unsafe_allow_html=True)

#load the model
# classifier = pickle.load(open('final_pipeline.pkl','rb'))


# customer segmentation function
def segment_customers(input_data):
    
    prediction=final_pipeline.predict(pd.DataFrame(input_data, columns=['State', 'Market', 'Market Size', 'Product Type', 'Type']))
    output_str = prediction[0]
    return output_str

def main():
    
   
    State = st.text_input("Type In The State",type='default')
    Market = st.radio ( "Select Market", ('East', 'West','South','Central') )
    Market_Size = st.radio ( "Select Market Size", ('Small Market', 'Major Market') )
    Product_Type = st.radio ( "Select Product Type", ("Coffee","Tea","Espresso","Herbal Tea") )
    Type = st.radio ( "Please Select The Type", ('Regular', 'Decaf') )
    
    
    result = ""

    # when button is clicked, make the prediction and store it
    if st.button("Suggest Product"):
        result=segment_customers([[State,Market,Market_Size,Product_Type,Type]])
    
    st.success(result)
    

if __name__ == '__main__':
        main ()